<a href="https://colab.research.google.com/github/georgecrossv/Assignments/blob/master/Copy_of_Langchain_with_PDF_using_cassio_Law_cases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Langchain Retrieval Augmentation (using Law data)
Large Language Models (LLMs) have a data freshness problem. The most powerful LLMs in the world, like GPT-4, have no idea about recent world events.

The world of LLMs is frozen in time. Their world exists as a static snapshot of the world as it was within their training data.

A solution to this problem is retrieval augmentation. The idea behind this is that we retrieve relevant information from an external knowledge base and give that information to our LLM. In this notebook we will learn how to do that. In this demo, external or proprietary data will be stored in Astra DB and used to provide more current LLM responses.

## Colab-specific setup

Make sure you have a Database and get ready to upload the Secure Connect Bundle and supply the Token string
(see [Pre-requisites](https://cassio.org/start_here/#vector-database) on cassio.org for details).

Likewise, ensure you have the necessary secret for the LLM provider of your choice: you'll be asked to input it shortly
(see [Pre-requisites](https://cassio.org/start_here/#llm-access) on cassio.org for details).

_Note: some portions of this notebook is part of the CassIO documentation. Visit [this page on cassIO.org](https://cassio.org/frameworks/langchain/qa-basic/)._


In [ ]:
# install required dependencies
! pip install \
    "langchain" \
    "cassandra-driver>=3.28.0" \
    "cassio" \
    "google-cloud-aiplatform>=1.25.0" \
    "jupyter>=1.0.0" \
    "openai==0.27.7" \
    "tiktoken==0.4.0" \
    "pypdf" \
    "transformers" \
    "huggingface_hub" \
    "sentence_transformers" \
    "unstructured"

You will likely be asked to "Restart the Runtime" at this time, as some dependencies
have been upgraded. **Please do restart the runtime now** for a smoother execution from this point onward.

#Load a PDF file

In [5]:
# Upload your PDF document:
import os
from google.colab import files


print('Please upload a PDF to train')
uploaded = files.upload()
i = 0
while i < len(uploaded):
    pdfFileTitle = list(uploaded.keys())[i]
    PDF_PATH = os.path.join(os.getcwd(), pdfFileTitle)
    # print("pdf path is: " + PDF_PATH)
    i = i + 1
    if i < len(uploaded):
      break
else:
    raise ValueError(
        'Cannot proceed loading a PDF file. Please re-run the cell.'
    )

Please upload a PDF to train


Saving McCall v. Microsoft Corp., 241 F. Supp. 2d 563 (2003).pdf to McCall v. Microsoft Corp., 241 F. Supp. 2d 563 (2003).pdf
pdf path is: /content/McCall v. Microsoft Corp., 241 F. Supp. 2d 563 (2003).pdf


ValueError: ignored

In [10]:
from langchain.document_loaders.unstructured import UnstructuredAPIFileIOLoader
from langchain.document_loaders import PyPDFLoader

pdf_folder_path = f'/content/Data'
os.listdir(pdf_folder_path)

loaders = [UnstructuredAPIFileIOLoader(os.path.join(pdf_folder_path, fn)) for fn in os.listdir(pdf_folder_path)]
loaders

# loader = PyPDFLoader(PDF_PATH)
#pages = loader.load_and_split()

In [ ]:
pages[0]

# Configure the Astra DB Connection

In [11]:
# Input your database keyspace name:
ASTRA_DB_KEYSPACE = input('Your Astra DB Keyspace name: ')

Your Astra DB Keyspace name: vs_law


In [ ]:
# Input your Astra DB token string, the one starting with "AstraCS:..."
ASTRA_DB_TOKEN_BASED_PASSWORD = input('Your Astra DB Token: ')

### Astra DB Secure Connect Bundle

Please upload the Secure Connect Bundle zipfile to connect to your Astra DB instance.

The Secure Connect Bundle is needed to establish a secure connection to the database.
Click [here](https://awesome-astra.github.io/docs/pages/astra/download-scb/#c-procedure) for instructions on how to download it from Astra DB.

In [13]:
# Upload your Secure Connect Bundle zipfile:
import os
from google.colab import files

print('Please upload your Secure Connect Bundle')
uploaded = files.upload()
if uploaded:
    astraBundleFileTitle = list(uploaded.keys())[0]
    ASTRA_DB_SECURE_BUNDLE_PATH = os.path.join(os.getcwd(), astraBundleFileTitle)
else:
    raise ValueError(
        'Cannot proceed without Secure Connect Bundle. Please re-run the cell.'
    )

Please upload your Secure Connect Bundle


Saving secure-connect-law.zip to secure-connect-law.zip


In [14]:
# colab-specific override of helper functions
from cassandra.cluster import (
    Cluster,
)
from cassandra.auth import PlainTextAuthProvider

# The "username" is the literal string 'token' for this connection mode:
ASTRA_DB_TOKEN_BASED_USERNAME = 'token'


def getCQLSession(mode='astra_db'):
    if mode == 'astra_db':
        cluster = Cluster(
            cloud={
                "secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH,
            },
            auth_provider=PlainTextAuthProvider(
                ASTRA_DB_TOKEN_BASED_USERNAME,
                ASTRA_DB_TOKEN_BASED_PASSWORD,
            ),
        )
        astraSession = cluster.connect()
        return astraSession
    else:
        raise ValueError('Unsupported CQL Session mode')

def getCQLKeyspace(mode='astra_db'):
    if mode == 'astra_db':
        return ASTRA_DB_KEYSPACE
    else:
        raise ValueError('Unsupported CQL Session mode')

### LLM Provider

In the cell below you can choose between **GCP VertexAI** or **OpenAI** for your LLM services.
(See [Pre-requisites](https://cassio.org/start_here/#llm-access) on cassio.org for more details).

Make sure you set the `llmProvider` variable and supply the corresponding access secrets in the following cell.

In [63]:
# Set your secret(s) for LLM access:
llmProvider = input(f'Please pick between "HuggingFace", "OpenAI", or "GCP_VertexAI": ') #' HuggingFace' # 'OpenAI'   # 'GCP_VertexAI'
#llmProvider = 'OpenAI'

Please pick between "HuggingFace", "OpenAI", or "GCP_VertexAI": OpenAI


In [ ]:
if llmProvider == 'OpenAI':
    apiSecret = input(f'Your secret for LLM provider "{llmProvider}": ')
    os.environ['OPENAI_API_KEY'] = apiSecret
elif llmProvider == 'GCP_VertexAI':
    # we need a json file
    print(f'Please upload your Service Account JSON for the LLM provider "{llmProvider}":')
    from google.colab import files
    uploaded = files.upload()
    if uploaded:
        vertexAIJsonFileTitle = list(uploaded.keys())[0]
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join(os.getcwd(), vertexAIJsonFileTitle)
    else:
      raise ValueError(
            'No file uploaded. Please re-run the cell.'
            )
elif llmProvider == 'HuggingFace':
    apiSecret = input(f'Your API for Hugging Face "{llmProvider}": ')
    os.environ['HUGGINGFACEHUB_API_TOKEN'] = apiSecret
else:
    raise ValueError('Unknown/unsupported LLM Provider')

### Colab preamble completed

The following cells constitute the demo notebook proper.

Test - using Facebook AI Search

In [ ]:
!pip install faiss-cpu

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings())
docs = faiss_index.similarity_search("What is Account Management?", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:300])

# Vector Similarity Search QA Quickstart

_**NOTE:** this uses Cassandra's "Vector Similarity Search" capability.
Make sure you are connecting to a vector-enabled database for this demo._

In [17]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

The following line imports the Cassandra flavor of a LangChain vector store:

In [18]:
from langchain.vectorstores.cassandra import Cassandra

A database connection is needed to access Cassandra. The following assumes
that a _vector-search-capable Astra DB instance_ is available. Adjust as needed.

In [19]:
# creation of the DB connection
cqlMode = 'astra_db'
session = getCQLSession(mode=cqlMode)
keyspace = getCQLKeyspace(mode=cqlMode)

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(139036530720832) 3d123259-cbab-4680-96fe-063ce8fc2e8a-us-east1.db.astra.datastax.com:29042:7930db73-72b7-4f36-9969-954647aba210> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


Both an LLM and an embedding function are required.

Below is the logic to instantiate the LLM and embeddings of choice. We choose to leave it in the notebooks for clarity.

In [65]:
# creation of the LLM resources

if llmProvider == 'GCP_VertexAI':
    from langchain.llms import VertexAI
    from langchain.embeddings import VertexAIEmbeddings
    llm = VertexAI()
    myEmbedding = VertexAIEmbeddings()
    print('LLM+embeddings from VertexAI')
elif llmProvider == 'OpenAI':
    from langchain.llms import OpenAI
    from langchain.embeddings import OpenAIEmbeddings
    llm = OpenAI(temperature=0)
    myEmbedding = OpenAIEmbeddings()
    print('LLM+embeddings from OpenAI')
elif llmProvider == 'HuggingFace':
    from langchain.llms import HuggingFaceHub
    from langchain.embeddings import HuggingFaceEmbeddings
    myEmbedding = HuggingFaceEmbeddings()
    repo_id = "databricks/dolly-v2-3b"
    llm = HuggingFaceHub(repo_id=repo_id, model_kwargs={"temperature": 0.7, "max_length": 500})
else:
    raise ValueError('Unknown LLM provider.')

LLM+embeddings from OpenAI


## Langchain Retrieval Augmentation

The following is a minimal usage of the Cassandra vector store. The store is created and filled at once, and is then queried to retrieve relevant parts of the indexed text, which are then stuffed into a prompt finally used to answer a question.

The following creates an "index creator", which knows about the type of vector store, the embedding to use and how to preprocess the input text:

_(Note: stores built with different embedding functions will need different tables. This is why we append the `llmProvider` name to the table name in the next cell.)_

In [60]:
table_name = 'vs_law_pdf_' + llmProvider

index_creator = VectorstoreIndexCreator(
    vectorstore_cls=Cassandra,
    embedding=myEmbedding,
    text_splitter=CharacterTextSplitter(
        chunk_size=400,
        chunk_overlap=0,
    ),
    vectorstore_kwargs={
        'session': session,
        'keyspace': keyspace,
        'table_name': table_name,
    },
)

Create the Cassandra Vector Store and clear entries if the table already exists

In [66]:
myCassandraVStore = Cassandra(
    embedding=myEmbedding,
    session=session,
    keyspace=keyspace,
    table_name=table_name,
)
# clears here
myCassandraVStore.clear()

In [36]:
mySplitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=120)

Create the function for creating a vector index for a Wikipedia entry

In [37]:
def create_vector_index_for_page(page, myCassandraVStore):
  page_chunks = mySplitter.transform_documents(page)
  myCassandraVStore.add_documents(page_chunks)

Execute the create_vector_index function for each row in the Wikipedia dataframe. It's good time to grab a drink as the next step will take about 90 seconds to complete.

In [42]:
# loader = PyPDFLoader(PDF_PATH)
# pages = loader.load_and_split()



In [53]:
##### Function to add pages

# /content/McCall v. Microsoft Corp., 241 F. Supp. 2d 563 (2003).pdf



def insert_and_add_pdf(number):

  PDF_PATH = pdf_folder_path + "/" + os.listdir(pdf_folder_path)[number]
  loader = PyPDFLoader(PDF_PATH)
  pages = loader.load_and_split()
  print(len(pages))
  for page in pages:
    page_chunks = mySplitter.transform_documents([page])
    myCassandraVStore.add_documents(page_chunks)




In [54]:
for i in range(len(os.listdir(pdf_folder_path))):
  insert_and_add_pdf(i)

10
4


In [ ]:
print(len(pages))

4


In [ ]:
for page in pages:
  page_chunks = mySplitter.transform_documents([page])
  myCassandraVStore.add_documents(page_chunks)

In [55]:
index = VectorStoreIndexWrapper(vectorstore=myCassandraVStore)

Now let's query our proprietory store. We'll ask "What is Andouille?"

In [ ]:
query = "What is Account Management?"
index.query(query,llm=llm)

' Account Management is the process of establishing and maintaining a strong relationship with a customer.\n\nQuestion:'

I'm really interested in what temperature to cook my andouille.

In [68]:
query = "What is the background of both cases?"
index.query(query,llm=llm)


" The first case is about determining whether the court erred in denying Kandler's motion to suppress evidence from a warrantless blood test. Kandler contends that her consent to the blood test was not valid. The second case is about Microsoft Corp. and plaintiffs' claims for unjust enrichment under common law. The opinions of the Oklahoma Court of Appeals and the Maryland Court of Appeals are not precedential and are merely persuasive in nature."

In [71]:
query = "Who were the key parties involved in Kandler case?"
index.query(query,llm=llm)

' The key parties involved in the Kandler case were Joanne Kandler (Petitioner) and the City of Kent (Respondent).'

In [72]:
query = "Who were the plaintiffs and who were the defendants?"
index.query(query,llm=llm)

' The plaintiffs were Gregory Paukstis, Christopher Furnas, Strickley v. Microsoft Corp., Prentice v. Microsoft Corp., and Kentucky v. Microsoft Corp. The defendant was Microsoft Corp.'

In [75]:
query = "Say I had  case where I tried to suppress evidence from a warrantless finger test.  Is there any case that also had a warrantless test done?"
index.query(query,llm=llm)

' No, Reichenbach applies only to breath tests and was inapplicable here, where a blood test was at issue.'

Let's compare this answer to what OpenAi GPT-3 will return

In [ ]:
import openai

openai.api_key = apiSecret
response = openai.Completion.create(
  engine="text-davinci-003",
  prompt="What is the background of the McCall v. Microsoft Corp. case?",
  max_tokens=100
)

print(response.choices[0].text.strip())

You've now seen how we can use a LLM to answer the prompt from our Astra Vector Store, but notice that the answer is different from using the LLM directly.

Let's get some information about the source for the response to the question "What temperature should Andouille be cooked?"

In [ ]:
retriever = index.vectorstore.as_retriever(search_kwargs={
    'k': 2,
})

In [ ]:
retriever.get_relevant_documents(
    "What is the background of the McCall v. Microsoft Corp. case?"
)

[Document(page_content='violations.\nLitig.,\n127\nCorp.\nAntitrust\nre\nMicrosoft\n (D.Md.2001).\n702,\nMi-\n722-23\nF.Supp.2d\nMaryland\n—\nMcCall\nupon\nis\npresent\nmotion\nbased\ncrosoft’s\nCorp.,\nv.\nIn\nDavidson\nMicrosoft\nthat\ncourt\ndecisions\nhave\nappellate\nstate\n336,\n344,\n43,\ncert.', metadata={'page': '2.0', 'source': '/content/McCall v. Microsoft Corp., 241 F. Supp. 2d 563 (2003) (1).pdf'}),
 Document(page_content='other\n data\nhighest\nthat\nthe\ncourt\nof\nthe\nstate\n6.\nThe\nclaims\nin\nv.\nMcCall\nMicrosoft\nFeiock,\nwould\ndecide\notherwise.”\nHicks\nv.\nCorp.,\nother\nthan\ninjunctive\nthe\nclaims\nfor\n624,\n3,\n1423,\n485\nU.S.\n630\nn.\n108\nS.Ct.\n99\nrelief,\nare\ndismissed.\n (1988);', metadata={'page': '3.0', 'source': '/content/McCall v. Microsoft Corp., 241 F. Supp. 2d 563 (2003) (1).pdf'})]